# Regress QAP measures against scanning parameters

In [6]:
# Load some needed packages
library(grid)
library(gridExtra)
library(ggplot2)
library(dplyr)
library(repr)
library(reshape2)
library(tidyr)
library(lme4)
library(utils)
library(multcomp)
library(xtable)

## Utility functions

In [ ]:

timestamp_to_numeric <- function(x) {
    times=unlist(strsplit(as.character(x), ":"))
    out_time = NaN
    
    if ( length(times) > 0 )
    {
        out_time <- as.numeric(times[1])*60.0
    }
    if ( length(times) > 1 )
    {
        out_time <- out_time + as.numeric(times[2])
    }
    if ( length(times) > 2 )
    {
        out_time <- out_time + as.numeric(times[3])/100.0
    }
    return(out_time)
}

# sMRI DATA

# 1. Read in and tidy the sMRI scanning parameters for ABIDE

In [189]:
abide_smri_scan_params<-read.csv("anat_abide_scan_params_bids.csv")

# reduce to just the Siemens systems
abide_smri_scan_params<-abide_smri_scan_params %>% 
                     filter(Manufacturer == "Siemens") %>%
                         mutate(Scanner = paste(Manufacturer,ManufacturersModelName))
abide_smri_scan_params$Scanner<-factor(abide_smri_scan_params$Scanner)

abide_smri_scan_params<-abide_smri_scan_params %>% 
                     rowwise() %>% 
                       mutate(VoxelVolume = as.numeric(SliceThickness)*
                                            prod(as.numeric(unlist(strsplit(as.character(PixelSpacing), "x"))))) 

abide_smri_scan_params<-abide_smri_scan_params %>% 
                     rowwise() %>%
                       mutate(AcquisitionDuration = timestamp_to_numeric(AcquisitionDuration))

abide_smri_scan_params<-abide_smri_scan_params %>% 
                     rowwise() %>%
                       mutate(AcquisitionDurationCalc = RepetitionTime * NumberOfSlices)

abide_smri_scan_params<-abide_smri_scan_params %>% 
                      dplyr::select(Site,Scanner,EchoTime,RepetitionTime,InversionTime,FlipAngle,VoxelVolume) %>% 
                      drop_na()

abide_smri_scan_params$Scanner<-droplevels(abide_smri_scan_params$Scanner)

In [190]:
abide_smri_scan_params %>% tbl_df()

Site,Scanner,EchoTime,RepetitionTime,InversionTime,FlipAngle,VoxelVolume
CMU_a,Siemens Verio,0.002480,1.87,1.100,8,1.000
CMU_b,Siemens Verio,0.002480,1.87,1.100,8,1.000
Caltech,Siemens TrioTim,0.002730,1.59,0.800,10,1.000
MaxMun_a,Siemens Verio,0.003060,1.80,0.900,9,1.000
MaxMun_b,Siemens Verio,0.003060,1.80,0.900,9,1.000
MaxMun_c,Siemens Verio,0.003060,1.80,0.900,9,1.000
MaxMun_d,Siemens Verio,0.003060,1.80,0.900,9,1.000
NYU,Siemens Allegra,0.003250,2.53,1.100,7,1.690
OHSU,Siemens TrioTim,0.003589,2.30,0.900,10,1.100
Olin,Siemens Allegra,0.002740,2.50,0.900,8,1.000


## 2. Read in and tidy the sMRI QAP measures for ABIDE

In [191]:
abide_anat_spat_df<-read.csv("2016_05_ABIDE_qap_anatomical_spatial.csv")

id.vars=c('Participant','Site','Session','Series')
measure.vars=c('CNR','Cortical.Contrast','EFC','FBER','FWHM','Qi1','SNR')
abide_anat_spat_df<-abide_anat_spat_df[c(id.vars,measure.vars)]

# reduce to just session_1 and anat_1 and remove rows with missing values
abide_anat_spat_df <- abide_anat_spat_df %>% 
                        filter(Session == "session_1" & Series == "anat_1") %>% 
                           drop_na()

# make sure that participant is a factor
abide_anat_spat_df$Participant <- factor(abide_anat_spat_df$Participant)

# summary(abide_anat_spat_df)

# plots
qap_label_strings=c(CNR='CNR',
                Cortical.Contrast='Cortical Contrast',
                EFC='EFC',
                FBER='FBER',
                FWHM='Smoothness (FWHM)',
                Qi1='Fraction of Artifact Voxels',
                SNR='SNR')

# combine with parameters
abide_anat_df <- inner_join(abide_anat_spat_df, abide_smri_scan_params, by="Site")

In [192]:
abide_anat_df %>% tbl_df()

Participant,Site,Session,Series,CNR,Cortical.Contrast,EFC,FBER,FWHM,Qi1,SNR,Scanner,EchoTime,RepetitionTime,InversionTime,FlipAngle,VoxelVolume
50002,Pitt,session_1,anat_1,11.737636,0.3916667,0.5026067,249.3866,3.845912,0.15565057,17.75640,Siemens Allegra,0.00393,2.1,1,7,1.331
50003,Pitt,session_1,anat_1,9.307147,0.3205829,0.4786799,194.6180,4.246237,0.15879603,16.71580,Siemens Allegra,0.00393,2.1,1,7,1.331
50004,Pitt,session_1,anat_1,12.188784,0.3945931,0.4730299,264.1895,3.837011,0.16217479,18.77359,Siemens Allegra,0.00393,2.1,1,7,1.331
50005,Pitt,session_1,anat_1,12.461291,0.3810753,0.4629147,274.4577,3.633874,0.11558958,19.43004,Siemens Allegra,0.00393,2.1,1,7,1.331
50006,Pitt,session_1,anat_1,13.148553,0.3893785,0.4227490,469.4240,3.633893,0.12287874,22.24350,Siemens Allegra,0.00393,2.1,1,7,1.331
50007,Pitt,session_1,anat_1,9.443249,0.3565112,0.4976299,206.1733,4.094131,0.22746124,15.76759,Siemens Allegra,0.00393,2.1,1,7,1.331
50008,Pitt,session_1,anat_1,12.116583,0.3923378,0.4710950,273.5004,3.754357,0.09592457,18.49452,Siemens Allegra,0.00393,2.1,1,7,1.331
50009,Pitt,session_1,anat_1,11.007108,0.4003765,0.5061345,223.2074,3.520997,0.18468458,17.44546,Siemens Allegra,0.00393,2.1,1,7,1.331
50010,Pitt,session_1,anat_1,13.123298,0.3815857,0.4074926,371.6116,3.769759,0.07750221,20.05090,Siemens Allegra,0.00393,2.1,1,7,1.331
50011,Pitt,session_1,anat_1,11.428044,0.4132088,0.5134501,204.3232,3.697480,0.13473372,16.36881,Siemens Allegra,0.00393,2.1,1,7,1.331


## 3. Read in and tidy the sMRI scanning parameters for CORR

In [193]:
corr_smri_scan_params<-read.csv("anat_corr_scan_params.csv")

corr_smri_scan_params<-corr_smri_scan_params %>%
                     filter(Manufacturer == "Siemens") %>%
                         mutate(Scanner = paste(Manufacturer,ManufacturersModelName))
corr_smri_scan_params$Scanner<-factor(corr_smri_scan_params$Scanner)

corr_smri_scan_params<-corr_smri_scan_params %>% 
                     rowwise() %>% 
                       mutate(VoxelVolume = as.numeric(SliceThickness)*
                                            prod(as.numeric(unlist(strsplit(as.character(PixelSpacing), "x")))))

corr_smri_scan_params<-corr_smri_scan_params %>%
                     filter(PulseSequenceType != "3D MP2RAGE") %>%
                       filter(PulseSequenceType != "3D MEMPR")

corr_smri_scan_params<-corr_smri_scan_params %>% 
                     dplyr::select(Site,Scanner,EchoTime,RepetitionTime,InversionTime,FlipAngle,VoxelVolume) %>% 
                       drop_na()


# tidy up columns, dropping unused levels and converting cols to numeric
for( i in colnames(corr_smri_scan_params)){
    if (( i != "Site" ) & ( i != "Scanner" )){
        if (! is.numeric(corr_smri_scan_params[[i]])){
            corr_smri_scan_params[[i]]<-as.numeric(levels(corr_smri_scan_params[[i]]))[corr_smri_scan_params[[i]]]
        }
    }
}

# corr_smri_scan_params$Site<-droplevels(corr_smri_scan_params$Site)
# corr_smri_scan_params$Scanner<-droplevels(corr_smri_scan_params$Scanner)

corr_smri_scan_params %>% tbl_df()

Warning message:
“NAs introduced by coercion”Warning message:
“NAs introduced by coercion”Warning message:
“NAs introduced by coercion”

Site,Scanner,EchoTime,RepetitionTime,InversionTime,FlipAngle,VoxelVolume
BMB 1,Siemens TrioTim,0.00298,2.30,0.90,9,1.000000
BNU 1,Siemens TrioTim,0.00339,2.53,1.10,7,1.690000
BNU 2 ses-1,Siemens TrioTim,0.00339,2.53,1.10,7,1.690000
BNU 2 ses-2,Siemens TrioTim,0.00345,2.53,1.10,7,1.000000
BNU 3,Siemens TrioTim,0.00339,2.53,1.10,7,1.729000
IBA TRT,Siemens TrioTim,0.00302,2.60,0.90,8,1.000000
IPCAS 1,Siemens TrioTim,0.00251,2.53,1.10,7,1.300000
IPCAS 2,Siemens TrioTim,0.00295,2.30,0.90,9,1.054688
IPCAS 3,Siemens TrioTim,0.00251,2.53,1.10,7,1.330000
IPCAS 5,Siemens TrioTim,0.00350,2.53,1.10,7,1.000000


## 4. Read in and tidy the sMRI QAP measures for CORR

In [15]:
corr_anat_spat_df<-read.csv("2016_05_CORR_qap_anatomical_spatial.csv")

id.vars=c('Participant','Site','Session','Series')
measure.vars=c('CNR','Cortical.Contrast','EFC','FBER','FWHM','Qi1','SNR')
corr_anat_spat_df<-corr_anat_spat_df[c(id.vars,measure.vars)]

# reduce to just session_1 and anat_1 and remove rows with missing values
corr_anat_spat_df <- corr_anat_spat_df %>% 
                        filter(Session == "session_1" & Series == "anat_1") %>% 
                           drop_na()

# make sure that participant is a factor
corr_anat_spat_df$Participant <- factor(corr_anat_spat_df$Participant)

# summary(abide_anat_spat_df)

# plots
qap_label_strings=c(CNR='CNR',
                Cortical.Contrast='Cortical Contrast',
                EFC='EFC',
                FBER='FBER',
                FWHM='Smoothness (FWHM)',
                Qi1='Fraction of Artifact Voxels',
                SNR='SNR')

corr_anat_spat_df <- inner_join(corr_anat_spat_df, corr_smri_scan_params, by="Site")

Warning message in inner_join_impl(x, y, by$x, by$y, suffix$x, suffix$y):
“joining factors with different levels, coercing to character vector”

In [16]:
corr_anat_spat_df %>% tbl_df()

Participant,Site,Session,Series,CNR,Cortical.Contrast,EFC,FBER,FWHM,Qi1,SNR,Scanner,EchoTime,RepetitionTime,InversionTime,FlipAngle,VoxelVolume
25599,JHNU,session_1,anat_1,10.595293,0.3195716,0.4407884,516.3453,3.51983,0.08878368,26.09653,Siemens TrioTim,0.00298,2.3,0.9,9,1
25600,JHNU,session_1,anat_1,10.988521,0.3188174,0.4454495,677.4726,4.20135,0.09439833,26.58296,Siemens TrioTim,0.00298,2.3,0.9,9,1
25601,JHNU,session_1,anat_1,12.270253,0.3321005,0.4481019,863.2795,3.61285,0.11224674,28.71940,Siemens TrioTim,0.00298,2.3,0.9,9,1
25602,JHNU,session_1,anat_1,10.025403,0.3190558,0.5021002,855.3761,3.12180,0.19547611,29.61605,Siemens TrioTim,0.00298,2.3,0.9,9,1
25603,JHNU,session_1,anat_1,12.544036,0.3787305,0.4084452,530.8371,3.33419,0.10272382,25.69635,Siemens TrioTim,0.00298,2.3,0.9,9,1
25604,JHNU,session_1,anat_1,12.152581,0.3327836,0.4175791,403.3878,4.69805,0.10388854,25.79123,Siemens TrioTim,0.00298,2.3,0.9,9,1
25605,JHNU,session_1,anat_1,13.688974,0.3797904,0.4062813,578.4140,3.06218,0.09054422,26.51364,Siemens TrioTim,0.00298,2.3,0.9,9,1
25606,JHNU,session_1,anat_1,12.730425,0.3468796,0.4465486,656.8925,3.71535,0.10712243,28.04013,Siemens TrioTim,0.00298,2.3,0.9,9,1
25607,JHNU,session_1,anat_1,11.528671,0.3245080,0.4311167,523.4361,3.61318,0.08763413,27.32128,Siemens TrioTim,0.00298,2.3,0.9,9,1
25608,JHNU,session_1,anat_1,11.826949,0.3242923,0.4250187,608.4486,4.16382,0.11716681,27.40298,Siemens TrioTim,0.00298,2.3,0.9,9,1


## 5. Analyze!

In [17]:
# Combine the twodatasets into one large dataframe
qap_param_df<-bind_rows(abide_anat_df,corr_anat_spat_df)

# mean center all of the covariates
qap_param_df<-qap_param_df %>% mutate_if(is.numeric, funs(scale(., scale=TRUE)))


Warning message in bind_rows_(x, .id):
“Unequal factor levels: coercing to character”Warning message in bind_rows_(x, .id):
“binding factor and character vector, coercing into character vector”Warning message in bind_rows_(x, .id):
“Unequal factor levels: coercing to character”Warning message in bind_rows_(x, .id):
“Unequal factor levels: coercing to character”Warning message in bind_rows_(x, .id):
“Unequal factor levels: coercing to character”

In [194]:
meas_stats=data.frame()
for ( meas in measure.vars ){
    print(meas)
    full_formula=as.formula(
        sprintf("%s ~ 0 + Scanner + EchoTime + InversionTime + RepetitionTime + FlipAngle + VoxelVolume",
                  meas))
    reduced_formula=as.formula(
        sprintf("%s ~ 0 + EchoTime + InversionTime + RepetitionTime + FlipAngle + VoxelVolume",
                  meas))
    full_model<-lm(full_formula, qap_param_df)
    reduced_model<-lm(reduced_formula, qap_param_df)
    scanner_f=anova(full_model,reduced_model)
    all_t=summary(full_model)
    
    meas_stats <- rbind(meas_stats,
                       data.frame(Measure=meas,
                          covariate="Scanner",
                          stat=scanner_f$F[2],
                          val_p=scanner_f[2,"Pr(>F)"]))
    
    for( covariate in c('EchoTime', 'InversionTime', 'RepetitionTime', 'FlipAngle', 'VoxelVolume')){
            meas_stats <- rbind(meas_stats,
                       data.frame(Measure=meas,
                          covariate=covariate,
                          stat=all_t$coefficients[covariate,'t value'],
                          val_p=all_t$coefficients[covariate,'Pr(>|t|)']))
    }
    
}

meas_stats$val_p <- p.adjust(meas_stats$val_p, "BH")
# smri_qap_param_model_red<-lm(SNR ~ 0 + EchoTime + InversionTime + 
#                                RepetitionTime + FlipAngle + VoxelVolume,
#                                qap_param_df)
# # summary(smri_qap_param_model)
# # anova(smri_qap_param_model,smri_qap_param_model_red)

[1] "CNR"
[1] "Cortical.Contrast"
[1] "EFC"
[1] "FBER"
[1] "FWHM"
[1] "Qi1"
[1] "SNR"


In [140]:
meas_stats = meas_stats %>% 
    gather(mes, value, -Measure, -covariate) %>% 
        mutate( cov_stat = paste(covariate, mes, sep="_")) %>% 
            dplyr::select(Measure, cov_stat, value) %>%
                spread(cov_stat, value)

In [141]:
meas_stats %>%tbl_df()

Measure,EchoTime_stat,EchoTime_val_p,FlipAngle_stat,FlipAngle_val_p,InversionTime_stat,InversionTime_val_p,RepetitionTime_stat,RepetitionTime_val_p,Scanner_stat,Scanner_val_p,VoxelVolume_stat,VoxelVolume_val_p
CNR,1.418004,1.730561e-01,-9.384289,1.247423e-19,3.87476134,1.425093e-04,-8.607014,6.782255e-17,41.82401,1.032657e-24,-4.3652618,1.882541e-05
Cortical.Contrast,4.737734,3.573231e-06,-13.563284,1.838982e-37,-14.64388421,1.055955e-42,18.034716,1.352785e-60,34.02688,2.260130e-20,7.9970424,7.389488e-15
EFC,4.579905,7.294963e-06,6.577464,1.287629e-10,0.01713395,9.863340e-01,1.809167,8.036319e-02,31.36848,6.313651e-19,-4.8582139,2.060004e-06
FBER,-15.038849,1.126026e-44,3.914693,1.249080e-04,26.39859789,1.219142e-109,-20.984189,3.004838e-77,122.94146,1.379259e-64,-17.3007857,1.410534e-56
FWHM,-1.140737,2.738800e-01,-3.443972,7.232608e-04,0.58162879,5.890277e-01,-7.371662,6.485370e-13,21.69937,2.577427e-13,0.4089238,6.993552e-01
Qi1,9.476575,5.928352e-20,9.328483,1.917401e-19,14.37695358,2.137408e-41,-21.910161,1.393153e-82,38.04655,1.298559e-22,-13.8132914,1.206296e-38
SNR,-9.724294,7.614931e-21,-3.157919,1.922636e-03,18.34820288,2.724213e-62,-18.531834,2.992323e-63,92.05821,1.365529e-50,-15.4270729,1.226371e-46


In [142]:
print(xtable(meas_stats, digits=3, display=c('s','s','g','g','g','g','g','g','g','g','g','g','g','g')))

% latex table generated in R 3.3.2 by xtable 1.8-2 package
% Thu Mar 23 17:53:32 2017
\begin{table}[ht]
\centering
\begin{tabular}{rlrrrrrrrrrrrr}
  \hline
 & Measure & EchoTime\_stat & EchoTime\_val\_p & FlipAngle\_stat & FlipAngle\_val\_p & InversionTime\_stat & InversionTime\_val\_p & RepetitionTime\_stat & RepetitionTime\_val\_p & Scanner\_stat & Scanner\_val\_p & VoxelVolume\_stat & VoxelVolume\_val\_p \\ 
  \hline
1 & CNR & 1.42 & 0.173 & -9.38 & 1.25e-19 & 3.87 & 0.000143 & -8.61 & 6.78e-17 & 41.8 & 1.03e-24 & -4.37 & 1.88e-05 \\ 
  2 & Cortical.Contrast & 4.74 & 3.57e-06 & -13.6 & 1.84e-37 & -14.6 & 1.06e-42 &   18 & 1.35e-60 &   34 & 2.26e-20 &    8 & 7.39e-15 \\ 
  3 & EFC & 4.58 & 7.29e-06 & 6.58 & 1.29e-10 & 0.0171 & 0.986 & 1.81 & 0.0804 & 31.4 & 6.31e-19 & -4.86 & 2.06e-06 \\ 
  4 & FBER &  -15 & 1.13e-44 & 3.91 & 0.000125 & 26.4 & 1.22e-109 &  -21 & 3e-77 &  123 & 1.38e-64 & -17.3 & 1.41e-56 \\ 
  5 & FWHM & -1.14 & 0.274 & -3.44 & 0.000723 & 0.582 & 0.589 & -7.37 & 6.49

# fMRI DATA

## 1. Read in and tidy the fMRI acquisition parameters for ABIDE

In [209]:
abide_fmri_scan_params<-read.csv("rest_abide_scan_params_bids_slicetimes.csv")

# reduce to just the Siemens systems
abide_fmri_scan_params<-abide_fmri_scan_params %>% 
                     filter(Manufacturer == "Siemens") %>%
                         mutate(Scanner = paste(Manufacturer,ManufacturersModelName))
abide_fmri_scan_params$Scanner<-factor(abide_fmri_scan_params$Scanner)
       
abide_fmri_scan_params<-abide_fmri_scan_params %>% 
                     rowwise() %>% 
                       mutate(VoxelVolume = as.numeric(SliceThickness)*
                                            prod(as.numeric(unlist(strsplit(as.character(PixelSpacing), "x"))))) 

abide_fmri_scan_params<-abide_fmri_scan_params %>% 
                     rowwise() %>%
                       mutate(AcquisitionDuration = timestamp_to_numeric(AcquisitionDuration))

abide_fmri_scan_params<-abide_fmri_scan_params %>% 
                      dplyr::select(Site, Scanner, AcquisitionDuration, EchoTime ,RepetitionTime, FlipAngle, VoxelVolume) %>% 
                      drop_na()

abide_fmri_scan_params$Scanner<-droplevels(abide_fmri_scan_params$Scanner)

In [210]:
abide_fmri_scan_params %>% tbl_df()

Site,Scanner,AcquisitionDuration,EchoTime,RepetitionTime,FlipAngle,VoxelVolume
CMU_a,Siemens Verio,486,0.030,2.0,73,27.000
CMU_b,Siemens Verio,486,0.030,1.5,73,27.000
Caltech,Siemens TrioTim,304,0.030,2.0,75,42.875
MaxMun_a,Siemens Verio,366,0.030,3.0,80,36.000
MaxMun_b,Siemens Verio,366,0.030,3.0,80,36.000
MaxMun_c,Siemens Verio,366,0.030,3.0,80,27.000
MaxMun_d,Siemens Verio,606,0.030,3.0,80,27.000
NYU,Siemens Allegra,360,0.015,2.0,90,36.000
OHSU,Siemens TrioTim,212,0.030,2.5,90,54.872
Olin,Siemens Allegra,315,0.027,1.5,60,46.240


## 2. Read in and tidy the fMRI QAP measures for ABIDE

In [211]:
abide_func_spat_df<-read.csv("2016_05_ABIDE_qap_functional_spatial.csv")

id.vars=c('Participant','Site','Session','Series')
measure.vars.spat=c('EFC','FBER','FWHM','Ghost_y','SNR')

# reduce to just the columns that we are interested in
abide_func_spat_df<-abide_func_spat_df[c(id.vars,measure.vars.spat)]

abide_func_temp_df<-read.csv("2016_05_ABIDE_qap_functional_temporal.csv")

id.vars=c('Participant','Site','Session','Series')
measure.vars.temp=c('Fraction.of.Outliers..Mean.','GCOR','Quality..Mean.',
               'RMSD..Mean.', 'Std..DVARS..Mean.')

# reduce to just the columns that we are interested in
abide_func_temp_df<-abide_func_temp_df[c(id.vars, measure.vars.temp)]

# join the spatial and temporal measures
abide_func_df<-inner_join(abide_func_spat_df, abide_func_temp_df)


# reduce to just session_1 and anat_1 and remove rows with missing values
abide_func_df <- abide_func_df %>% 
                     filter(Session == "session_1" & Series == "rest_1") %>% 
                         droplevels() %>%
                             drop_na()

# make sure that participant is a factor
abide_func_df$Participant <- factor(abide_func_df$Participant)


# plots
qap_label_strings=c(Ghost_y='GSR',
                EFC='EFC',
                FBER='FBER',
                FWHM='Smoothness (FWHM)',
                SNR='SNR',
                Fraction.of.Outliers..Mean.='Mean % Outliers',
                GCOR='GCOR',
                Quality..Mean.='Mean Quality',
                RMSD..Mean.='Mean RMSD',
                Std..DVARS..Mean.="Mean Std. Dvars.")

# combine with parameters
abide_func_df <- inner_join(abide_func_df, abide_fmri_scan_params, by="Site")


summary(abide_func_df)

Joining, by = c("Participant", "Site", "Session", "Series")


  Participant       Site          Session       Series         EFC        
 50002  :  1   NYU    :184   session_1:692   rest_1:692   Min.   :0.4185  
 50003  :  1   USM    :101                                1st Qu.:0.4984  
 50004  :  1   UCLA_1 : 81                                Median :0.5276  
 50005  :  1   Pitt   : 57                                Mean   :0.5294  
 50006  :  1   Yale   : 56                                3rd Qu.:0.5596  
 50007  :  1   Caltech: 38                                Max.   :0.6704  
 (Other):686   (Other):175                                                
      FBER             FWHM          Ghost_y                SNR        
 Min.   : 8.141   Min.   :1.576   Min.   :-0.0164149   Min.   : 6.273  
 1st Qu.:21.985   1st Qu.:1.811   1st Qu.: 0.0005513   1st Qu.: 9.837  
 Median :26.633   Median :1.915   Median : 0.0349412   Median :10.819  
 Mean   :27.635   Mean   :1.977   Mean   : 0.0299075   Mean   :10.991  
 3rd Qu.:32.769   3rd Qu.:2.066   3rd Qu

## 3. Read in and tidy the fMRI acquisition parameters for CoRR

In [214]:
corr_fmri_scan_params<-read.csv("func_corr_scan_params.csv")

# reduce to just the Siemens systems
corr_fmri_scan_params<-corr_fmri_scan_params %>% 
                     filter(Manufacturer == "Siemens") %>%
                         mutate(Scanner = paste(Manufacturer,ManufacturersModelName))
corr_fmri_scan_params$Scanner<-factor(corr_fmri_scan_params$Scanner)

        
corr_fmri_scan_params<-corr_fmri_scan_params %>% 
                     rowwise() %>% 
                       mutate(VoxelVolume = as.numeric(SliceThickness)*
                                            prod(as.numeric(unlist(strsplit(as.character(PixelSpacing), "x"))))) 

corr_fmri_scan_params<-corr_fmri_scan_params %>% 
                     rowwise() %>%
                       mutate(AcquisitionDuration = timestamp_to_numeric(AcquisitionDuration))

corr_fmri_scan_params<-corr_fmri_scan_params %>% 
                      dplyr::select(Site, Scanner, AcquisitionDuration, EchoTime ,RepetitionTime, FlipAngle, VoxelVolume) %>% 
                      drop_na()

corr_fmri_scan_params$Scanner<-droplevels(corr_fmri_scan_params$Scanner)

In [215]:
summary(corr_fmri_scan_params)

                   Site                Scanner   AcquisitionDuration
 NKI TRT acq-tr1400ms: 4   Siemens Allegra : 2   Min.   :122.0      
 NKI TRT acq-tr645ms : 3   Siemens Magentom: 2   1st Qu.:352.0      
 IBA_TRT             : 2   Siemens TrioTim :34   Median :465.0      
 BMB 1               : 1   Siemens Verio   : 1   Mean   :431.2      
 BNU 1               : 1                         3rd Qu.:494.0      
 BNU 2 ses-1         : 1                         Max.   :915.0      
 (Other)             :27                                            
    EchoTime      RepetitionTime    FlipAngle      VoxelVolume    
 Min.   :0.0150   Min.   :0.645   Min.   :60.00   Min.   : 0.512  
 1st Qu.:0.0300   1st Qu.:1.750   1st Qu.:72.50   1st Qu.:27.000  
 Median :0.0300   Median :2.000   Median :90.00   Median :34.680  
 Mean   :0.0289   Mean   :2.009   Mean   :82.18   Mean   :32.691  
 3rd Qu.:0.0300   3rd Qu.:2.150   3rd Qu.:90.00   3rd Qu.:40.028  
 Max.   :0.0300   Max.   :4.000   Max.   :90.0

## 4. Read in and tidy the fMRI QAP measures for CoRR

In [216]:
corr_func_spat_df<-read.csv("2016_05_CoRR_qap_functional_spatial.csv")

id.vars=c('Participant','Site','Session','Series')

# reduce to just the columns that we are interested in
corr_func_spat_df<-corr_func_spat_df[c(id.vars,measure.vars.spat)]

corr_func_temp_df<-read.csv("2016_05_CORR_qap_functional_temporal.csv")


# reduce to just the columns that we are interested in
corr_func_temp_df<-corr_func_temp_df[c(id.vars, measure.vars.temp)]

# join the spatial and temporal measures
corr_func_df<-inner_join(corr_func_spat_df, corr_func_temp_df)


# reduce to just session_1 and anat_1 and remove rows with missing values
corr_func_df <- corr_func_df %>% 
                     filter(Session == "session_1" & Series == "rest_1") %>% 
                         droplevels() %>%
                             drop_na()

# make sure that participant is a factor
corr_func_df$Participant <- factor(corr_func_df$Participant)


# plots
qap_label_strings=c(Ghost_y='GSR',
                EFC='EFC',
                FBER='FBER',
                FWHM='Smoothness (FWHM)',
                SNR='SNR',
                Fraction.of.Outliers..Mean.='Mean % Outliers',
                GCOR='GCOR',
                Quality..Mean.='Mean Quality',
                RMSD..Mean.='Mean RMSD',
                Std..DVARS..Mean.="Mean Std. Dvars.")

# combine with parameters
corr_func_df <- inner_join(corr_func_df, corr_fmri_scan_params, by="Site")


summary(corr_func_df)

Joining, by = c("Participant", "Site", "Session", "Series")
Warning message in inner_join_impl(x, y, by$x, by$y, suffix$x, suffix$y):
“joining factors with different levels, coercing to character vector”

  Participant      Site                Session       Series         EFC        
 27223  :  2   Length:250         session_1:250   rest_1:250   Min.   :0.3816  
 27224  :  2   Class :character                                1st Qu.:0.4205  
 27225  :  2   Mode  :character                                Median :0.4585  
 27226  :  2                                                   Mean   :0.4688  
 27227  :  2                                                   3rd Qu.:0.5121  
 27228  :  2                                                   Max.   :0.6053  
 (Other):238                                                                   
      FBER            FWHM          Ghost_y               SNR        
 Min.   :12.78   Min.   :1.640   Min.   :-0.005613   Min.   : 8.536  
 1st Qu.:20.50   1st Qu.:1.873   1st Qu.: 0.018513   1st Qu.:10.925  
 Median :24.42   Median :1.976   Median : 0.023375   Median :12.022  
 Mean   :24.84   Mean   :1.972   Mean   : 0.025588   Mean   :12.180  
 3rd Qu.:2

## 5. Analyze!

In [217]:
# Combine the twodatasets into one large dataframe
qap_param_df<-bind_rows(abide_func_df,corr_func_spat_df)

# mean center all of the covariates
qap_param_df<-qap_param_df %>% mutate_if(is.numeric, funs(scale(., scale=TRUE)))

Warning message in bind_rows_(x, .id):
“Unequal factor levels: coercing to character”Warning message in bind_rows_(x, .id):
“Unequal factor levels: coercing to character”Warning message in bind_rows_(x, .id):
“Unequal factor levels: coercing to character”Warning message in bind_rows_(x, .id):
“Unequal factor levels: coercing to character”

In [203]:
qap_param_df

Participant,Site,Session,Series,EFC,FBER,FWHM,Ghost_y,SNR,Fraction.of.Outliers..Mean.,GCOR,Quality..Mean.,RMSD..Mean.,Std..DVARS..Mean.,Scanner,AcquisitionDuration,EchoTime,RepetitionTime,FlipAngle,VoxelVolume
50002,Pitt,session_1,rest_1,1.47377133,-0.01385772,-0.50218953,0.998439378,-0.106669225,-0.8013943,-0.8698341,-0.89192640,-0.206601871,-0.13278639,Siemens Allegra,-0.9600646,0.09596658,-1.353209,-1.093522,0.1186266
50003,Pitt,session_1,rest_1,0.96379411,-0.01385759,-0.05448966,0.503054741,-0.085882238,-0.6937101,-0.8667884,-0.44974995,0.872790938,-0.69583159,Siemens Allegra,-0.9600646,0.09596658,-1.353209,-1.093522,0.1186266
50004,Pitt,session_1,rest_1,0.98154742,-0.01385721,-0.51973548,0.682490033,-0.077662417,1.8380488,-0.8161337,-0.53721653,-0.066484897,1.55996963,Siemens Allegra,-0.9600646,0.09596658,-1.353209,-1.093522,0.1186266
50005,Pitt,session_1,rest_1,0.63890681,-0.01385724,-0.37015482,-0.002368603,-0.061692146,-0.5915727,-0.8335101,-0.46933533,-0.040888296,0.28232800,Siemens Allegra,-0.9600646,0.09596658,-1.353209,-1.093522,0.1186266
50006,Pitt,session_1,rest_1,0.73697402,-0.01385682,0.53787150,0.984277246,-0.053682195,-0.5019714,-0.8414969,-0.41723312,-0.401070301,0.37218124,Siemens Allegra,-0.9600646,0.09596658,-1.353209,-1.093522,0.1186266
50007,Pitt,session_1,rest_1,1.40143283,-0.01385674,-0.63141636,1.026926411,-0.072152576,-0.6017484,-0.7469466,-0.58694522,0.103067537,-0.07545324,Siemens Allegra,-0.9600646,0.09596658,-1.353209,-1.093522,0.1186266
50008,Pitt,session_1,rest_1,0.86018776,-0.01385740,-0.37723387,1.006136805,-0.082103961,-0.6605079,-0.8076164,-0.46311233,0.261722150,-0.35515382,Siemens Allegra,-0.9600646,0.09596658,-1.353209,-1.093522,0.1186266
50009,Pitt,session_1,rest_1,1.23419488,-0.01385700,-0.04957923,0.878903439,-0.074574871,-0.4163302,-0.8634816,-0.71512284,0.274690903,1.04388777,Siemens Allegra,-0.9600646,0.09596658,-1.353209,-1.093522,0.1186266
50010,Pitt,session_1,rest_1,-0.30991437,-0.01385760,-0.03703717,0.323274884,-0.039454824,-0.7202115,-0.8782309,0.22924641,-0.128150630,0.44281621,Siemens Allegra,-0.9600646,0.09596658,-1.353209,-1.093522,0.1186266
50011,Pitt,session_1,rest_1,1.19405215,-0.01385642,-0.02354602,0.829860120,-0.042798309,-0.7503846,-0.8933540,-0.57634444,-0.431468666,0.28373556,Siemens Allegra,-0.9600646,0.09596658,-1.353209,-1.093522,0.1186266


In [218]:
meas_stats=data.frame()
measure.vars=c(measure.vars.spat, measure.vars.temp)
for ( meas in measure.vars ){
    print(meas)
    full_formula=as.formula(
        sprintf("%s ~ 0 + Scanner + EchoTime + RepetitionTime + FlipAngle + VoxelVolume",
                  meas))
    reduced_formula=as.formula(
        sprintf("%s ~ 0 + EchoTime + RepetitionTime + FlipAngle + VoxelVolume",
                  meas))
    full_model<-lm(full_formula, qap_param_df)
    reduced_model<-lm(reduced_formula, qap_param_df)
    scanner_f=anova(full_model,reduced_model)
    all_t=summary(full_model)
    
    meas_stats <- rbind(meas_stats,
                       data.frame(Measure=meas,
                          covariate="Scanner",
                          stat=scanner_f$F[2],
                          val_p=scanner_f[2,"Pr(>F)"]))
    
    for( covariate in c('EchoTime', 'RepetitionTime', 'FlipAngle', 'VoxelVolume')){
            meas_stats <- rbind(meas_stats,
                       data.frame(Measure=meas,
                          covariate=covariate,
                          stat=all_t$coefficients[covariate,'t value'],
                          val_p=all_t$coefficients[covariate,'Pr(>|t|)']))
    }
    
}

meas_stats$val_p <- p.adjust(meas_stats$val_p, "BH")
# smri_qap_param_model_red<-lm(SNR ~ 0 + EchoTime + InversionTime + 
#                                RepetitionTime + FlipAngle + VoxelVolume,
#                                qap_param_df)
# # summary(smri_qap_param_model)
# # anova(smri_qap_param_model,smri_qap_param_model_red)

[1] "EFC"
[1] "FBER"
[1] "FWHM"
[1] "Ghost_y"
[1] "SNR"
[1] "Fraction.of.Outliers..Mean."
[1] "GCOR"
[1] "Quality..Mean."
[1] "RMSD..Mean."
[1] "Std..DVARS..Mean."


In [219]:
meas_stats = meas_stats %>% 
    gather(mes, value, -Measure, -covariate) %>% 
        mutate( cov_stat = paste(covariate, mes, sep="_")) %>% 
            dplyr::select(Measure, cov_stat, value) %>%
                spread(cov_stat, value)

In [220]:
meas_stats %>% tbl_df()

Measure,EchoTime_stat,EchoTime_val_p,FlipAngle_stat,FlipAngle_val_p,RepetitionTime_stat,RepetitionTime_val_p,Scanner_stat,Scanner_val_p,VoxelVolume_stat,VoxelVolume_val_p
EFC,13.754052,1.336055e-37,-5.849484,1.232003e-08,8.1936891,2.491276e-15,1.122550e+02,2.221127e-58,-22.28261186,3.845292e-82
FBER,-2.374903,2.073021e-02,2.821346,6.150967e-03,4.0078408,9.713840e-05,1.432801e+11,0.000000e+00,1.62235385,1.168753e-01
FWHM,-18.526318,1.261025e-61,-23.785381,1.570914e-90,10.1849640,2.322621e-22,1.961074e+02,1.438225e-90,3.21937860,1.724780e-03
Ghost_y,20.155494,1.936884e-70,-15.891455,5.526106e-48,1.3293640,1.959260e-01,1.959804e+01,6.020816e-12,-11.23322263,1.597880e-26
SNR,-7.773324,5.399918e-14,10.040259,7.979241e-22,-2.8084838,6.243073e-03,5.675585e+01,2.980316e-32,12.85361307,1.779466e-33
Fraction.of.Outliers..Mean.,-6.096606,3.015487e-09,-9.029344,3.934606e-18,5.7713006,1.862566e-08,2.230548e+01,1.602586e-13,0.01060031,9.915454e-01
GCOR,-40.923330,9.450498e-185,-21.346374,5.624446e-77,4.6209551,6.915883e-06,5.362295e+01,1.202523e-30,9.05759315,3.271965e-18
Quality..Mean.,-17.737224,1.639631e-57,10.706646,2.137059e-24,-3.2194972,1.724780e-03,8.412095e+01,1.005625e-45,8.19350390,2.491276e-15
RMSD..Mean.,4.552246,9.235782e-06,-1.868116,7.065066e-02,-0.8316739,4.227946e-01,7.526710e-01,5.316464e-01,-3.93379190,1.279674e-04
Std..DVARS..Mean.,9.005337,4.577658e-18,6.491042,2.821079e-10,1.5146860,1.416448e-01,3.467936e+00,1.897236e-02,3.25330524,1.617185e-03


In [222]:
print(xtable(meas_stats, digits=3, display=c('s','s','g','g','g','g','g','g','g','g','g','g')))

% latex table generated in R 3.3.2 by xtable 1.8-2 package
% Mon Mar 27 16:24:47 2017
\begin{table}[ht]
\centering
\begin{tabular}{rlrrrrrrrrrr}
  \hline
 & Measure & EchoTime\_stat & EchoTime\_val\_p & FlipAngle\_stat & FlipAngle\_val\_p & RepetitionTime\_stat & RepetitionTime\_val\_p & Scanner\_stat & Scanner\_val\_p & VoxelVolume\_stat & VoxelVolume\_val\_p \\ 
  \hline
1 & EFC & 13.8 & 1.34e-37 & -5.85 & 1.23e-08 & 8.19 & 2.49e-15 &  112 & 2.22e-58 & -22.3 & 3.85e-82 \\ 
  2 & FBER & -2.37 & 0.0207 & 2.82 & 0.00615 & 4.01 & 9.71e-05 & 1.43e+11 &    0 & 1.62 & 0.117 \\ 
  3 & FWHM & -18.5 & 1.26e-61 & -23.8 & 1.57e-90 & 10.2 & 2.32e-22 &  196 & 1.44e-90 & 3.22 & 0.00172 \\ 
  4 & Ghost\_y & 20.2 & 1.94e-70 & -15.9 & 5.53e-48 & 1.33 & 0.196 & 19.6 & 6.02e-12 & -11.2 & 1.6e-26 \\ 
  5 & SNR & -7.77 & 5.4e-14 &   10 & 7.98e-22 & -2.81 & 0.00624 & 56.8 & 2.98e-32 & 12.9 & 1.78e-33 \\ 
  6 & Fraction.of.Outliers..Mean. & -6.1 & 3.02e-09 & -9.03 & 3.93e-18 & 5.77 & 1.86e-08 & 22.3 & 1.6e-